In [1]:
pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install qiskit-optimization

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
start = time.time()

import pandas as pd
import numpy as np


# Read the datasets
weather = pd.read_csv('kalai_weather.csv')
weather_test = pd.read_csv('weather.csv')
car = pd.read_csv('kalai_car.csv')

# Preprocessing
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

combined_weather_list = list(weather.Mon_weather) + list(weather.Tue_weather) \
+ list(weather.Wed_weather) + list(weather.Thu_weather) + list(weather.Fri_weather) + list(weather.Sat_weather) + list(weather.Sun_weather)

weather_onehot = enc.fit_transform(np.asarray(combined_weather_list).reshape(-1,1))

timestep_dict = dict(zip(weather.Monday, weather.timestep)) ## This will used to map the departure time

car['DepTime_Coded'] = car["Depature Time (End of Slot)"].map(timestep_dict)

car['diff_battery_kwh'] = (( car['Arrival Battery in %'] - car['Minimal Depature Battery in %'])/100) * 60
# Positive is discharge, and negative value means the charged

car['arrival_time'] = pd.to_datetime(car['Arrival Time'])
car['departure_time'] = pd.to_datetime(car['Depature Time (End of Slot)'])

discharge = car.diff_battery_kwh.apply(lambda a: a>=0)
recharge = car.diff_battery_kwh.apply(lambda a: a<0)

discharge_df = car[discharge].groupby(['Day_int', 'DepTime_Coded'])[['diff_battery_kwh']].sum().reset_index()
discharge_df.columns = ['Day', 'Timesteps', 'kWh']

recharge_df = car[recharge].groupby(['Day_int', 'DepTime_Coded'])[['diff_battery_kwh']].sum().reset_index()
recharge_df.columns = ['Day', 'Timesteps', 'kWh']

master_timestep = []
for i in range(1,8):
    for j in range(1,49):
        master_timestep.append([i,j])
        
master_timedf = pd.DataFrame(master_timestep)
master_timedf.columns = ['Day', 'Timesteps']

discharge_alltime_df = pd.merge(master_timedf, discharge_df, on=['Day', 'Timesteps'] , how='left')
recharge_alltime_df = pd.merge(master_timedf, recharge_df, on=['Day', 'Timesteps'] , how='left')
discharge_alltime_df['kWh'] = discharge_alltime_df['kWh'].fillna(0)
recharge_alltime_df['kWh'] = recharge_alltime_df['kWh'].fillna(0)

discharge_kwh = np.asarray(discharge_alltime_df['kWh']) ##arranged by timestep
recharge_kwh = np.asarray(recharge_alltime_df['kWh']) ## arranged by timestep

wk = []
for i in range(1,8):
    for j in range(1,49):
        if i < 6:
            wk.append(1)
        else:
            wk.append(2)
            
wk_onehot = enc.fit_transform(np.asarray(wk).reshape(-1,1))


# Classical Optimization Implementation
import gurobipy as gp
gurobipy_model = gp.Model("gurobi")
x = gurobipy_model.addVar(vtype=gp.GRB.INTEGER, name="solar", lb=0, ub=180)
y = gurobipy_model.addVar(vtype=gp.GRB.INTEGER, name="plug", lb=0)
z1 = gurobipy_model.addVar(vtype=gp.GRB.INTEGER, name="discharge", lb=0, ub=60)
z2 = gurobipy_model.addVar(vtype=gp.GRB.INTEGER, name="recharge", lb=0, ub=60)

gurobipy_model.setObjective(x + (y*405) + (z1*84) - (z2*84), gp.GRB.MINIMIZE)
gurobipy_model.addConstr(180 - x >= 0)
gurobipy_model.addConstr(x + z1 + y - z2 -120 >= 0)
gurobipy_model.addConstr(z1 - 59 <= 0)
gurobipy_model.addConstr(z2 - 17 >= 0)
gurobipy_model.update()
gurobipy_model.display()

from qiskit_optimization.translators import from_gurobipy
qp2 = from_gurobipy(gurobipy_model)
print(qp2.prettyprint())

# Classical Optimization
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer
gurobi_result = GurobiOptimizer().solve(qp2)
print("gurobi")
print(gurobi_result.prettyprint())

end = time.time()
print('The total runtime is', str(end - start)+' s')

Restricted license - for non-production use only - expires 2023-10-25
Minimize
  <gurobi.LinExpr: solar + 405.0 plug + 84.0 discharge + -84.0 recharge>
Subject To
  R0: <gurobi.LinExpr: -1.0 solar> >= -180
  R1: <gurobi.LinExpr: solar + plug + discharge + -1.0 recharge> >= 120
  R2: <gurobi.LinExpr: discharge> <= 59
  R3: <gurobi.LinExpr: recharge> >= 17
Bounds
  -0 <= solar <= 180
  -0 <= discharge <= 60
  -0 <= recharge <= 60
General Integers
  ['solar', 'plug', 'discharge', 'recharge']
Problem name: gurobi

Minimize
  84*discharge + 405*plug - 84*recharge + solar

Subject to
  Linear constraints (4)
    -solar >= -180  'R0'
    discharge + plug - recharge + solar >= 120  'R1'
    discharge <= 59  'R2'
    recharge >= 17  'R3'

  Integer variables (4)
    0 <= solar <= 180
    0 <= plug
    0 <= discharge <= 60
    0 <= recharge <= 60

gurobi
objective function value: -4860.0
variable values: solar=180.0, plug=-0.0, discharge=0.0, recharge=60.0
status: SUCCESS
The total runtime is 5.